### Les annonces de Renault Zoé en Île-de-France sur lacentrale.fr
On cherche les véhicules les plus intéressants par rapport à leur argus

On utilise selenium pour tromper les traceurs du site

In [127]:
# coding: utf-8
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import json

In [133]:
chromedriver = "/Applications/chromedriver"
driver = webdriver.Chrome(chromedriver)

prix = []
modele = []
annee = []
kilometrage = []
page = []

url = "https://www.lacentrale.fr/listing?makesModelsCommercialNames=RENAULT%3AZOE&options=&page={}&regions=FR-IDF"

for i in range(1, 15):
    driver.get(url.format(i))
    
    results = driver.find_elements_by_class_name('fieldPrice')
    prix.extend(map(lambda x: x.text, results))

    results = driver.find_elements_by_class_name('version')
    modele.extend(map(lambda x: x.text, results))
    
    results = driver.find_elements_by_class_name('fieldYear')
    annee.extend(map(lambda x: x.text, results))
    
    results = driver.find_elements_by_class_name('fieldMileage')
    kilometrage.extend(map(lambda x: x.text, results))
    
    page.extend([i for j in range(len(results))])
    
driver.close()

In [134]:
df = pd.DataFrame()
df["Prix"] = prix
df["Modèle"] = modele
df["Année"] = annee
df["Kilométrage"] = kilometrage
df["Page"] = page

df["Prix"] = pd.to_numeric(df["Prix"].str.replace('[^\d]', ''))
df["Kilométrage"] = df["Kilométrage"].str.replace('[^\d]', '')

print(df.iloc[:, 0:5])

        Prix                            Modèle Année Kilométrage  Page
0        NaN             R90 INTENS GAMME 2017  2017        7828     1
1        NaN                            INTENS  2014       17807     1
2     6990.0                 ZEN CHARGE RAPIDE  2014       46500     1
3     9200.0                          Q90 LIFE  2015       16461     1
4     9300.0          Q90 INTENS CHARGE RAPIDE  2015       34402     1
5    10700.0             Q90 ZEN CHARGE RAPIDE  2015       15153     1
6     8980.0            Q90 LIFE CHARGE RAPIDE  2014       21540     1
7     9100.0                              LIFE  2014       26664     1
8     9400.0                          Q90 LIFE  2015       20660     1
9     9200.0            Q90 LIFE CHARGE RAPIDE  2014       33086     1
10    8900.0            Q90 LIFE CHARGE RAPIDE  2015       14543     1
11    8490.0            Q90 LIFE CHARGE RAPIDE  2015       23802     1
12    7090.0                 ZEN CHARGE RAPIDE  2014       35246     1
13    

In [135]:
url_calcul = "https://www.lacentrale.fr/get_co_prox.php?km={}&zipcode=75014&year={}"

def get_argus(row):
    driver.get(url_calcul.format(row["Kilométrage"], row["Année"]))
    pre = driver.find_element_by_tag_name("pre").text
    data = json.loads(pre)
    return data["cote_perso"]

On approxime les argus de tous les modèles de Zoé à la Zoé Q90 Intens Charge Rapide Type 2 et on ne garde comme paramètres que le kilométrage et le millésime.

In [136]:
driver = webdriver.Chrome(chromedriver)
url_argus = "https://www.lacentrale.fr/cote-auto-renault-zoe-q90+intens+charge+rapide+type+2-2014.html"
driver.get(url_argus)

df["Argus"] = df.apply(get_argus, axis=1)

driver.close()

In [137]:
df["Delta"] = df["Argus"] - df["Prix"]
df.sort_values("Delta", ascending=False).head()

,Prix,Modèle,Année,Kilométrage,Page,Argus,Delta
63,5990.0,Q90 ZEN CHARGE RAPIDE,2014,19738,4,8908,2918.0
48,7440.0,Q90 LIFE CHARGE RAPIDE,2015,37449,4,9071,1631.0
13,7190.0,ZEN CHARGE RAPIDE,2014,25407,1,8811,1621.0
16,6990.0,ZEN CHARGE RAPIDE,2014,37633,2,8588,1598.0
12,7090.0,ZEN CHARGE RAPIDE,2014,35246,1,8646,1556.0


    Au 29/10/2018, l'occasion la plus intéressante est la Zoé Q90 Zen Charge Rapide de 2014 de la page 4 des résultats avec un prix de 2918€ en-dessous de l'argus.